# Добавление значений к состоянию цепочки

:::note

С этим руководством будет проще работать, если ознакомиться с разделами:

- [LangChain Expression Language (LCEL)](/docs/concepts/#langchain-expression-language)
- [Chaining runnables](/docs/how_to/sequence/)
- [Calling runnables in parallel](/docs/how_to/parallel/)
- [Custom functions](/docs/how_to/functions/)
- [Passing data through](/docs/how_to/passthrough)

:::

В качестве альтернативного сопособа [передачи данных внутри цепочки](/docs/how_to/passthrough) текущие значения состояния цепочки могут оставаться неизменными, в то время как новые значения сохраняются в соответствующих ключах.
Статический метод `RunnablePassthrough.assign(...)` принимает входное значение и добавляет аргументы, переданные в функцию assign.

Используйте метод для аддитивного создания словаря, который будет использоваться в качестве входных данных для последующего шага.
Такой подход часто используется при работе с [LCEL](/docs/concepts/#langchain-expression-language).

In [ ]:
%pip install --upgrade --quiet gigachain

In [2]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'extra': {'num': 1, 'mult': 3}, 'modified': 2}

Разберем представленный пример:

* На вход в цепочку передается значение `{"num": 1}`. Значение попадает в экземпляр `RunnableParallel`, который использует его при параллельном вызове Runnable-объектов.
* Вызывается поле `extra`. Метод `RunnablePassthrough.assign()` сохраняет первичное значение `{"num": 1}` в словаре и создает поле `mult` со значением `lambda x: x["num"] * 3)` (то есть `3`). Итоговый результат: `{"num": 1, "mult": 3}`.
* Значение `{"num": 1, "mult": 3}` возвращается в `RunnableParallel` и помещается в поле `exrtra`.
* В это же время вызывается поле `modified`. Значение поля — лямбда-функция, которая добавляет единицу к `"num"`. Поэтому `modified` принимает значение `2`.

Таким образом, результат работы примера — `{'extra': {'num': 1, 'mult': 3}, 'modified': 2}`.

## Обработка потоковой генерации

Отличительной особенностью метода является возможность передавать значения по мере их поступления.
Для демострации используем `RunnablePassthrough.assign()`, чтобы сразу возвращать исходные документы цепочки.

In [3]:
from langchain_community.chat_models.gigachat import GigaChat
from langchain_community.embeddings.gigachat import GigaChatEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"],
    embedding=GigaChatEmbeddings(
        credentials="<авторизационные_данные>", verify_ssl_certs=False
    ),
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = GigaChat(credentials="<авторизационные_данные>", verify_ssl_certs=False)

generation_chain = prompt | model | StrOutputParser()

retrieval_chain = {
    "context": retriever,
    "question": RunnablePassthrough(),
} | RunnablePassthrough.assign(output=generation_chain)

stream = retrieval_chain.stream("where did harrison work?")

for chunk in stream:
    print(chunk)

{'question': 'where did harrison work?'}
{'context': [Document(page_content='harrison worked at kensho')]}
{'output': ''}
{'output': 'H'}
{'output': 'arrison'}
{'output': ' worked'}
{'output': ' at'}
{'output': ' Kens'}
{'output': 'ho'}
{'output': '.'}
{'output': ''}


Так как оригинальный вопрос `"question"` доступен сразу после запуска цепочки, его можно будет найти в первом фрагменте.
Второй фрагмент содержит контекст `"context"`, так как ретривер вызывается после запуска цепочки.
Остальные фрагменты содержат результат потоковой генерации токенов.